Implementation of Successive Line Overrelaxation algorithm used for solving two-dimensional heat equation.
$$\frac{\partial \phi}{\partial t} = \frac{\partial}{\partial x}(\lambda_{x}\frac{\partial \phi}{\partial x}) + \frac{\partial}{\partial y}(\lambda_{y}\frac{\partial \phi}{\partial y})$$

In [1]:
import numpy as np

In [ ]:
class SLOR:
    def __init__(self, vx, vy, NX, NY, SizeX, SizeY, dt, t_max, eps=0.01):
        """
            vx, xy: lambda parameters of equation
            NX, NY: number of nodes of x y
            SizeX, SizeY: size of grid
            dt : time-step
            t_max: max_time
            eps: maximum convergence relative error
        """
        self.vx, self.vy = vx, vy
        self.NX, self.NY = NX, NY
        self.SizeX, self.SizeY = SizeX, SizeY
        self.dt = dt
        self.t_max = t_max
        self.eps = eps
        
    def convert(i , j):
        """
            This function maps indexes (i,j) to coordinates (x_i, y_i)
            0 < i,j < self.NX, self.NY
        """
        return float(i) / self.NX, float(j) / self.NY
    
    def solve(self, start_phi, border_phi):
        """
            start_phi: starting conditions, function from two parameters
            border_phi: border conditions, function from three conditions
        """
        # Creating grid
        dx = SizeX / NX
        dy = SizeY / NY
        # Coefficients
        TX = vx / dx ** 2
        TY = vy / dy ** 2
        
        b, c = np.full((self.NX, self.NY), -TX, dtype=float)
        f, g = np.full((self.NX, self.NY), -TY, dtype=float)
        gamma = 1. / self.dt
        d = np.zeros((self.NX, self.NY))
        a = -(c + b + g + f) + gamma
        for i in xrange(self.NX):
            for j in xrange(elf.NY):
                d = gamma * start_phi(convert(i, j))
        
        # Starting multilinear Gauss-Seidel method
        num_steps = float(self.t_max) / self.dt
        phi = np.zeros((num_steps + 1, self.NX, self.NY))
        # Set start_conditions
        for i in xrange(self.NX):
            for j in xrange(self.NY):
                phi[0][i][j] = start_phi(convert(i, j))
        for step in xrange(1, num_steps + 1):
            # Set border_conditions
            for i in xrange(self.NX):
                phi[step][i][0] = border_phi(convert(i, 0), step * self.dt)
                phi[step][i][self.NY - 1] = border_phi(convert(i, self.NY - 1), step * self.dt)
            for j in xrange(1, self.NY - 1):
                phi[step][0][j] = border_phi(convert(0, j), step * self.dt)
                phi[step][self.NX - 1][j] = border_phi(convert(self.NX - 1, j), step * self.dt)
            # Up
            # Thomas algorithm
            # p,q - auxillary variables
            # r - result vector in matrix equation: d[i][j] - g[i][j]*phi[step][i][j-1] - f[i][j]*phi[step][i][j + 1]
            p = q = r = np.zeros(self.NX)
            for j in xrange(1, self.NY - 1):
                for i in xrange(1, self.NX):
                    r[i] = d[i][j] - g[i][j]*phi[step][i][j - 1] - f[i][j]*phi[step - 1][i][j + 1]
                    p[i] = b[i][j] / a[i][j] if i == 0 else b[i][j] / (a[i][j] - c[i][j]*p[i - 1])
                    q[i] = r[i][j] / a[i][j] if i == 0 else (r[i][j] - c[i][j]) / (a[i][j] - c[i][j] * p[i - 1])
                phi[step][self.NX-1][j] = q[self.NX - 1]
                for i in xrange(self.NX-2, -1, -1):
                    phi[step][i][j] = q[i] - p[i] * phi[step][i + 1][j]
            
            # Down
            for j in xrange(self.NY-2, 0, -1):
                for i in xrange(1, self.NX):
                    r[i] = d[i][j] - g[i][j]*phi[step - 1][i][j - 1] - f[i][j]*phi[step][i][j + 1]
                    p[i] = b[i][j] / a[i][j] if i == 0 else b[i][j] / (a[i][j] - c[i][j]*p[i - 1])
                    q[i] = r[i][j] / a[i][j] if i == 0 else (r[i][j] - c[i][j]) / (a[i][j] - c[i][j] * p[i - 1])
                phi[step][self.NX-1][j] = q[self.NX - 1]
                for i in xrange(self.NX-2, -1, -1):
                    phi[step][i][j] = q[i] - p[i] * phi[step][i + 1][j]
                    
            p = q = r = np.zeros(self.NY)
            # Left
            for i in xrange(1, self.NX - 1):
                for j in xrange(1, self.NY):
                    r[j] = d[i][j] - c[i][j]*phi[step][i - 1][j] - b[i][j]*phi[step - 1][i + 1][j]
                    p[i] = f[i][j] / a[i][j] if j == 0 else f[i][j] / (a[i][j] - g[i][j]*p[i - 1])
                    q[i] = r[i][j] / a[i][j] if i == 0 else (r[i][j] - g[i][j]) / (a[i][j] - g[i][j] * p[i - 1])
                phi[step][i][self.NY-1] = q[self.NY - 1]
                for j in xrange(self.NY - 2, -1, -1):
                    phi[step][i][j] = q[i] - p[i] * phi[step][i][j + 1]
            # Right
            for i in xrange(self.NX-2, 0, -1):
                for j in xrange(1, self.NY):
                    r[j] = d[i][j] - c[i][j]*phi[step][i - 1][j] - b[i][j]*phi[step - 1][i + 1][j]
                    p[i] = f[i][j] / a[i][j] if j == 0 else f[i][j] / (a[i][j] - g[i][j]*p[i - 1])
                    q[i] = r[i][j] / a[i][j] if i == 0 else (r[i][j] - g[i][j]) / (a[i][j] - g[i][j] * p[i - 1])
                phi[step][i][self.NY-1] = q[self.NY - 1]
                for j in xrange(self.NY - 2, -1, -1):
                    phi[step][i][j] = q[i] - p[i] * phi[step][i][j + 1]
            # Checking for convergence